In [3]:
from src.common.vector import *
from src.common.reference_frame import *

r = Vector(-5368,-1784,3691)
(ra,dec) = radec_from_r(r)

print(f'RA = {np.rad2deg(ra)}\nDEC = {np.rad2deg(dec)}')

RA = 198.38370037548617
DEC = 33.12454287112769
